In [1]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
from IPython.display import display

def get_taxa_id(species_name):
    
    base_url = "http://api.inaturalist.org/v1/taxa/autocomplete?q="

    
    request = requests.get(base_url + "%20".join(species_name.split()))
    data = request.json()

    ids = []
    for i in data['results']:
        ids.append(i['id'])

    return ids

def get_observation(id_no, month, year):
    #Url builder, for the request
    base_url = "http://api.inaturalist.org/v1/observations?"
    end_url = "&order=desc&order_by=created_at"
    url = base_url + 'taxon_id=' + str(id_no) + '&month=' + str(month) + '&year=' + str(year) + end_url

    request = requests.get(url)
    data = request.json()    

    
    return data
    
        
def get_count_one_month(id_no_lst, month, year):
    
    count = 0
    for i in id_no_lst:
        count += int(get_observation(i, month, year)['total_results'])
    return count
    
    

species = [
    'Danaus plexippus',
    'Hyles lineata',
    'Zerene cesonia',
    'Papilio multicaudata',
    'Agraulis vanillae',
    'Papilio cresphontes',
    'Strymon melinus',
    'Vanessa cardui',
    'Hylephila phyleus',
    'Danaus gilippus'
]

months = [
    'January',
    'February',
    'March',
    'April',
    'May',
    'June',
    'July',
    'August',
    'September',
    'October',
    'November',
    'December'
]

def main():
    print('running')
    species_to_id = {}
    frames = []
    
    
    #Get a dictionary of the taxa -> lst(ids)
    for i in species:
        species_to_id[i] = get_taxa_id(i)
    
    month_map = dict(zip(range(1,13), months))
    
    species_dict_out = {}
    year = 2016
    
    print(species_to_id)
    
    #iterate through the 
    for spec in species_to_id:
        species_dict_out[spec] = {}
        
        for mon in month_map:
            species_dict_out[spec][month_map[mon]] = get_count_one_month(species_to_id[spec], mon, year)
        
        print(species_dict_out[spec])
        frames.append(species_dict_out[spec])
            
        
   
    result = json_normalize(frames)
    display(result)
    
    
main()
    

running
{'Vanessa cardui': [48548], 'Hyles lineata': [49348, 293806], 'Zerene cesonia': [129362, 239037, 532836], 'Hylephila phyleus': [50340, 312192], 'Papilio multicaudata': [68263, 237647, 148765, 460141, 545972], 'Agraulis vanillae': [49150, 312743, 461059, 416323, 208039, 234216], 'Strymon melinus': [50931, 346149, 238732, 238733, 542113, 238734, 238735], 'Danaus plexippus': [48662, 235550], 'Danaus gilippus': [51743, 51748, 50063, 51744, 51746, 51747, 51749], 'Papilio cresphontes': [50072]}
{'June': 97, 'November': 164, 'January': 12, 'August': 86, 'October': 104, 'May': 47, 'February': 24, 'December': 38, 'September': 76, 'March': 32, 'April': 61, 'July': 121}
{'June': 26, 'November': 5, 'January': 1, 'August': 70, 'October': 16, 'May': 43, 'February': 6, 'December': 1, 'September': 50, 'March': 73, 'April': 59, 'July': 67}
{'June': 5, 'November': 109, 'January': 3, 'August': 15, 'October': 91, 'May': 5, 'February': 1, 'December': 21, 'September': 57, 'March': 1, 'April': 7, 'Ju

,April,August,December,February,January,July,June,March,May,November,October,September
0,61,86,38,24,12,121,97,32,47,164,104,76
1,59,70,1,6,1,67,26,73,43,5,16,50
2,7,15,21,1,3,9,5,1,5,109,91,57
3,72,224,17,12,6,148,117,43,65,129,279,292
4,15,41,4,2,0,33,40,7,16,7,24,37
5,86,245,79,33,31,208,104,44,92,395,705,352
6,128,163,2,29,4,123,100,16,99,29,241,220
7,307,744,288,180,105,418,226,283,219,618,1269,986
8,23,130,86,12,18,93,58,16,37,251,372,210
9,9,104,3,5,1,54,40,10,18,12,76,73
